In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 102.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.9 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
-from langchain import PromptTemplate, LLMChain

template = """
        Detect named entities in following text delimited by double quotes.
        Return your response in json format with spans of named entities with fields "named entity","type","span".
        Return all entities
        `{text}`
        json format file:
"""

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)


# Modify the text to contain biomedical entities
text = """
The patient is a 55-year-old male with a history of hypertension and type 2 diabetes mellitus. He presents with a 2-day history of fever, cough, and shortness of breath. On examination, he is afebrile and has a heart rate of 120 beats per minute, blood pressure of 140/90 mmHg, and respiratory rate of 24 breaths per minute. He has diffuse crackles on lung examination. Chest X-ray shows bilateral infiltrates. The patient is admitted to the hospital for further evaluation and treatment.
"""

print(llm_chain.run(text))


        {
"named entity":["patient"],
"type":["person"],
"span":[{"start":0,"end":9},{"start":10,"end":19},{"start":20,"end":31}]
}
{
"named entity":["history"],
"type":["event"],
"span":[{"start":9,"end":19}]
}
{
"named entity":["fever"],
"type":["symptom"],
"span":[{"start":19,"end":24}]
}
{
"named entity":["cough"],
"type":["symptom"],
"span":[{"start":24,"end":31}]
}
{
"named entity":["shortness of breath"],
"type":["symptom"],
"span":[{"start":31,"end":40}]
}
{
"named entity":["heart rate"],
"type":["physiological_state"],
"span":[{"start":40,"end":48}]
}
{
"named entity":["blood pressure"],
"type":["physiological_state"],
"span":[{"start":48,"end":56}]
}
{
"named entity":["respiratory rate"],
"type":["physiological_state"],
"span":[{"start":56,"end":64}]
}
{
"named entity":["chest x-ray"],
"type":["diagnostic_test"],
"span":[{"start":64,"end":72}]
}
{
"named entity":["infiltrates"],
"type":["anatomical_structure"],
"span":[{"start":72,"end":80}]
}
        }

Expected output:

{
"